## Aproximate Nearest Neighbor from Scratch

Create Next : 
2. Add priority queue for predict function
3. based on max distance from every node

## Import Library

In [1]:
# import basic library for data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib

# import sys library 
# and append src folder into path
import sys
sys.path.append("../")

# import approximate_nn and knn library
from src.approximate_nn import ApproximateNearestNeighbor
from src.approximate_nn import KNearestNeighbor

# import train_test_split
from sklearn.model_selection import train_test_split

# import word embedding model
import tensorflow as tf
embedding = tf.keras.models.load_model("../../model/nnlm-id-dim50/")

2023-06-24 14:10:07.056225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 14:10:08.079811: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-24 14:10:08.122220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-24 14:10:08.122302: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

2023-06-24 14:10:11.408351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-24 14:10:11.408765: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-24 14:10:11.408913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SAT-PC): /proc/driver/nvidia/version does not exist
2023-06-24 14:10:11.410527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## ANN Before

In [47]:
class Tree:
    def __init__(self,
                 children_left = None,
                 children_right = None,
                 size : int = None,
                 is_leaf : bool = None,
                 hyperplane : np.array = None,
                 data_registered : np.array = None,
                 label_registered : np.array = None):
        """Class for store Node

        Args:
            children_left (Tree, optional): 
                Tree for data left. Defaults to None.
            children_right (_type_, optional): 
                Tree for data right. Defaults to None.
            size (int, optional): 
                Count of data point in a tree. Defaults to None.
        """
        self.children_left = children_left
        self.children_right = children_right
        self.size = size
        self.is_leaf = is_leaf
        self.hyperplane = hyperplane
        self.data_registered = data_registered
        self.label_registered = label_registered

In [48]:
class ApproximateNearestNeighborOld:
    def __init__(self,
                 min_sample_per_branch : int = 10,
                 distance_type : str = "eucledian"):
        self.min_sample_per_branch = min_sample_per_branch
        self.distance_type = distance_type

    def create_index(self,
                     X : np.array,
                     y : np.array) -> None :
        # Pass the data by value
        X = np.array(X).copy()
        y = np.array(y).copy()

        # Grow indexing Tree
        self.forest = self.create_forest(X, y, 10)

    def create_forest(self,
                    X : np.array,
                    y : np.array,
                    n_tree : int = 10) -> np.array :
        # prepare the container object
        trees = []

        for i in range(n_tree):
            tree = self.grow_index(X,y)
            trees.append(tree)
        
        return trees

    def grow_index(self,
                   X : np.array,
                   y : np.array,
                   depth : int = 0):
        # calculate node size
        node_size = len(X)

        # if the node size larger than minimum data
        if node_size > self.min_sample_per_branch:

            # create Tree object without
            # registering data
            node = Tree(size = node_size, is_leaf = False)

            # split the node
            # and calculate hyperplane parameter
            X_left, X_right, y_left, y_right, hyperplane = self.hyperplane_split(X, y)
            node.hyperplane = hyperplane

            # grow recursively the branch
            node.children_left = self.grow_index(X_left, y_left, depth+1)
            node.children_right = self.grow_index(X_right, y_right, depth+1)
        
        else :
            # create Tree object with
            # registered data
            node = Tree(size=node_size,
                        is_leaf = True,
                        data_registered = X,
                        label_registered = y,
                        hyperplane=None)
            
        return node
    
    def hyperplane_split(self,
                         X : np.array,
                         y :np.array):

        # pick 2 random data point from data using
        random_index = np.random.randint(low=0, high=len(X)-1, size=2)
        random_point = X[random_index]

        # calculate the middle point
        random_middle = np.sum(random_point, axis=0)/2

        # vector random point
        vector = random_point[0] - random_point[1]

        # Create the hyperplane equation
        c = -np.dot(vector,random_middle)
        hyperplane = np.append(vector, c)

        # calculate the sign for every data 
        result_sign = np.sign(np.dot(X, hyperplane[:-1].T) + hyperplane[-1])

        # separate the data into left and right data
        X_left = X[np.where(result_sign==-1)]
        X_right = X[np.where(result_sign==1)]
        y_left = y[np.where(result_sign==-1)]
        y_right = y[np.where(result_sign==1)]

        return X_left, X_right, y_left, y_right, hyperplane
    
    def find_items_forest(self,
                          X_in : np.array,
                          n_items : int = 10):

        # crate empty container for store 
        # item from each tree with feature 
        # of X plus 1 for y
        item_forest = np.empty((0,X_in.shape[1]+1), dtype=float)

        # populate data from leaf
        for tree in self.forest:
            item_tree = self.find_item_leaf(X_in, tree=tree)
            item_forest = np.append(item_forest, item_tree, axis=0)

        # eliminate duplicate data
        item_forest = np.unique(item_forest, axis=0)
        
        # separate the item feature and item_label
        X_forest = item_forest[:,:X_in.shape[1]].astype('float')
        y_forest = item_forest[:,-1]

        # rank the X_env and retrieve the y
        ranked_index = self.rank_neighbors(X_in,X_forest)
        
        
        if len(ranked_index) < n_items:
            return y_forest[ranked_index]
        else:
            return y_forest[ranked_index[:n_items]]
        
    def find_item_leaf(self,
                  X_in : np.array,
                  tree):
        
        if tree.is_leaf:
            return np.column_stack((tree.data_registered, tree.label_registered))
        
        else:
            sign_check = np.sign(np.dot(X_in, tree.hyperplane[:-1]) + tree.hyperplane[-1])
            if sign_check == 1:
                branch = tree.children_right
            else:
                branch = tree.children_left
            return self.find_item_leaf(X_in, branch)
        
    
    def rank_neighbors(self,
                         X_in : np.array,
                         X_env : np.array) -> np.array :
        """Method to compute distance between data input
           and registered data in a leaf, then rank the 
           environment

        Args:
            X_in (np.array) (k,):
                Data input into the environment with 1 
                data point and k number of feature
            X_env (np.array) (n,k):
                Data registered in the environment with
                n data point registered in the environment
                and k number of feature

        Returns:
            np.array (n,): 
                array of index of nearest item calculated 
                between the input and environment data 
                with size n data index
        """
        
        if self.distance_type == "eucledian":
            distance = np.linalg.norm(X_env - X_in, axis=1)
            rank_index = np.argsort(distance)

        elif self.distance_type == "manhattan":
            distance = np.sum(np.abs(X_env - X_in), axis=1)
            rank_index = np.argsort(distance)

        elif self.distance_type == "cosine-similarity":
            distance = np.squeeze(X_env @ X_in.T)/(np.linalg.norm(X_env, axis=1)*np.linalg.norm(X_in))
            rank_index = np.argsort(-distance)

        else:
            distance = np.squeeze(X_env @ X_in.T)
            rank_index = np.argsort(-distance)

        return rank_index

In [50]:
X = np.random.rand(100,5)
y = np.random.randint(low=100, high=1000, size=100)
y2 = y_train.sample(100, random_state=666).values.astype('str')

print(X[0], y[0])

[0.49424474 0.55992359 0.34487469 0.26941805 0.96149909] 347


In [51]:
model = ApproximateNearestNeighborOld(min_sample_per_branch=10)
model.create_index(X, y2)

In [43]:
model.find_items_forest(np.array([[0.19798236,0.35213506,0.34625828,0.71753865,0.53268257]]))

array(['leksikolog', 'neoimpresionisme', 'kanung', 'kenali asam bawah',
       'tebing penyamun', 'kuala', 'soakonora', 'manurungnge',
       'sondo-sondo', 'kaliajir'], dtype='<U32')

In [27]:
model.tree.children_left.children_left.children_left.children_right.hyperplane

None


## A. Data Preparation

In [2]:
# import embedding data
embedding_data = joblib.load("../../data/processed/word_embedding.pkl")
embedding_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93566 entries, 0 to 93565
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   words      93566 non-null  object
 1   embedding  93566 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [ ]:
## create train and test data
X = embedding_data['embedding']
y = embedding_data['words']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=666)

# save training and test data
joblib.dump(X_train, "../../data/processed/X_train.pkl")
joblib.dump(X_test, "../../data/processed/X_test.pkl")
joblib.dump(y_train, "../../data/processed/y_train.pkl")
joblib.dump(y_test, "../../data/processed/y_test.pkl")

In [3]:
# load pickle data
X_train = joblib.load("../../data/processed/X_train.pkl")
X_test = joblib.load("../../data/processed/X_test.pkl")
y_train = joblib.load("../../data/processed/y_train.pkl")
y_test = joblib.load("../../data/processed/y_test.pkl")

## B. Model Training / Registering Data Into Model

### B.1. ANN Model

In [11]:
ann_model = ApproximateNearestNeighbor(min_size_split=100, distance_type="cosine-similarity", n_tree = 5)
ann_model.fit(X = np.array(X_train.to_numpy().tolist()), y=y_train.values.astype('str'))

### B.2. KNN Model

In [5]:
knn_model = KNearestNeighbor(distance_type="cosine-similarity")
knn_model.fit(X = np.array(X_train.to_numpy().tolist()), y=y_train.values.astype('str'))

## C. Model Testing / Searching Similar Items

In [6]:
def search_similar_item(embedding_model : object,
                        neighbors_model : object,
                        text_input : str):
    # generate embedding for text input
    input_embed = embedding_model([text_input]).numpy().squeeze().tolist()

    # search neighbors by embedding 
    similar_item = neighbors_model.find_similar_items(np.array([input_embed]))

    return similar_item

### C.1. ANN Model

In [14]:
sim_item = search_similar_item(embedding_model=embedding, neighbors_model=ann_model, text_input="kucing")
sim_item

array(['kucing', 'angsa', 'ular', 'anjing', 'monyet', 'buaya', 'jerapah',
       'musang', 'kobra', 'koala'], dtype='<U44')

### C.2. KNN Model

In [15]:
sim_item_ = search_similar_item(embedding_model=embedding, neighbors_model=knn_model, text_input="kucing")
sim_item_

array(['kucing', 'angsa', 'ular', 'anjing', 'batu kucing', 'monyet',
       'kelinci', 'binatang', 'beruang', 'sumur kucing'], dtype='<U44')

## D. Searching Time Comparison

In [12]:
%%timeit
sim_item = search_similar_item(embedding_model=embedding, neighbors_model=ann_model, text_input="kucing")

28.5 ms ± 775 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
sim_item_ = search_similar_item(embedding_model=embedding, neighbors_model=knn_model, text_input="kucing")

29.5 ms ± 705 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
